In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
import datetime

In [ ]:
chernobyl_coords = (30.0927, 51.3870) # (Lon, lat) or (x, y)

MAP_SIZE_IN_DEGREES = (100, 50)

LOWER_LEFT_X_LON = chernobyl_coords[0] - MAP_SIZE_IN_DEGREES[0]/2
LOWER_LEFT_Y_LAT = chernobyl_coords[1] - MAP_SIZE_IN_DEGREES[1]/2
UPPER_LEFT_X_LON = chernobyl_coords[0] + MAP_SIZE_IN_DEGREES[0]/2
UPPER_LEFT_Y_LAT = chernobyl_coords[1] + MAP_SIZE_IN_DEGREES[1]/2



In [ ]:
fig = plt.figure(figsize=(12,9))

# world map has a lower left corner of (-180, -90) and an upper right corner of (180, 90)
m = Basemap(projection='mill',
            llcrnrlon=LOWER_LEFT_X_LON, # X axis for map is longitude
            llcrnrlat=LOWER_LEFT_Y_LAT, # Y axis for map is latitude
            urcrnrlon=UPPER_LEFT_X_LON, 
            urcrnrlat=UPPER_LEFT_Y_LAT,
            resolution="i") # c-crude, l-low, i-intermediate, h-high, f-full

m.shadedrelief() # makes it look pretty :)
m.drawcountries(linewidth=1.2)

# Locate Chernobyl
m.scatter(chernobyl_coords[0], chernobyl_coords[1], latlon=True, s=100, c='red', marker='^', alpha=1) # puts a red triangle at the (lon,lat) coordinate

plt.show()

In [ ]:
sensor_latitudes = []
sensor_longitudes = []

data = pd.read_csv("data/archive/data1.csv")
measurements = ["I_131_(Bq/m3)","Cs_134_(Bq/m3)","Cs_137_(Bq/m3)"]

data[measurements] = data[measurements].replace("?", "NaN")
data[measurements] = data[measurements].replace("L", "NaN")
data[measurements] = data[measurements].replace("N", "NaN")

data[measurements] = data[measurements].astype("float64")
data.dropna(inplace=True)

for lon, lat, ville in data.filter(['X','Y', "Ville"]).drop_duplicates().values:
    print(lon, lat, ville)
    sensor_longitudes.append(lon)
    sensor_latitudes.append(lat)


assert len(sensor_latitudes) == len(sensor_longitudes) and len(sensor_latitudes) > 50

In [ ]:
fig = plt.figure(figsize=(12,9))

# world map has a lower left corner of (-180, -90) and an upper right corner of (180, 90)
m = Basemap(projection='mill',
            llcrnrlon=LOWER_LEFT_X_LON, # X axis for map is longitude
            llcrnrlat=LOWER_LEFT_Y_LAT, # Y axis for map is latitude
            urcrnrlon=UPPER_LEFT_X_LON, 
            urcrnrlat=UPPER_LEFT_Y_LAT,
            resolution="i") # c-crude, l-low, i-intermediate, h-high, f-full

m.shadedrelief() # makes it look pretty :)
m.drawcountries(linewidth=1.2)
m.drawrivers(color="lightblue")

# Locate Chernobyl
m.scatter(chernobyl_coords[0], chernobyl_coords[1], latlon=True, s=100, c='red', marker='^', alpha=1) # puts a red triangle at the (lon,lat) coordinate

#Locate Each Sensor:
m.scatter(sensor_longitudes, sensor_latitudes, latlon=True, s=100, c='blue', marker='^', alpha=1) 



plt.show()

In [ ]:
colorbar = {
    'I_131_(Bq/m3)':{
        'min':0,
        'max':0
    },
    'Cs_134_(Bq/m3)':{
        'min':0,
        'max':0
    },
    'Cs_137_(Bq/m3)':{
        'min':0,
        'max':0
    },
}
v=[]
for lon, lat, ville, value in data.filter(['X','Y', "Ville", "I_131_(Bq/m3)"]).drop_duplicates().values:
    v.append(value)
colorbar['I_131_(Bq/m3)']['min'] = min(v)
colorbar['I_131_(Bq/m3)']['max'] = max(v)

v=[]
for lon, lat, ville, value in data.filter(['X','Y', "Ville", "Cs_134_(Bq/m3)"]).drop_duplicates().values:
    v.append(value)
colorbar['Cs_134_(Bq/m3)']['min'] = min(v)
colorbar['Cs_134_(Bq/m3)']['max'] = max(v)

v=[]
for lon, lat, ville, value in data.filter(['X','Y', "Ville", "Cs_137_(Bq/m3)"]).drop_duplicates().values:
    v.append(value)
colorbar['Cs_137_(Bq/m3)']['min'] = min(v)
colorbar['Cs_137_(Bq/m3)']['max'] = max(v)

In [ ]:
# data = pd.read_csv("data/archive/data1.csv")
def create_map(date, iso_widget):
    sensor_latitudes = []
    sensor_longitudes = []
    iso_values = []


    selected_date = date.strftime("%y") + "/" + date.strftime("%m") + "/" + date.strftime("%d")

    for lon, lat, ville, value in data[data.Date == selected_date].filter(['X','Y', "Ville", iso_widget]).drop_duplicates().values:
        # print(lon, lat, ville)
        sensor_longitudes.append(lon)
        sensor_latitudes.append(lat)
        iso_values.append(float(value))

    # assert len(sensor_latitudes) == len(sensor_longitudes) and len(sensor_latitudes) > 50
    fig = plt.figure(figsize=(12,9))

    # world map has a lower left corner of (-180, -90) and an upper right corner of (180, 90)
    m = Basemap(projection='mill',
                llcrnrlon=LOWER_LEFT_X_LON, # X axis for map is longitude
                llcrnrlat=LOWER_LEFT_Y_LAT, # Y axis for map is latitude
                urcrnrlon=UPPER_LEFT_X_LON,
                urcrnrlat=UPPER_LEFT_Y_LAT,
                resolution="i") # c-crude, l-low, i-intermediate, h-high, f-full

    m.shadedrelief() # makes it look pretty :)
    m.drawcountries(linewidth=1.2)

    # Locate Chernobyl
    m.scatter(chernobyl_coords[0], chernobyl_coords[1], latlon=True, s=100, c='red', marker='^', alpha=1) # puts a red triangle at the (lon,lat) coordinate

    #Locate Each Sensor:
    # print(iso_values)
    sc = m.scatter(sensor_longitudes, sensor_latitudes, latlon=True, s=100, c=iso_values, cmap='hot', marker='^', alpha=1, vmin=colorbar[iso_widget]['min'], vmax=colorbar[iso_widget]['max']) 
    plt.colorbar(sc)
    
    plt.show()

In [ ]:
date_widget = widgets.DatePicker()
date_widget.value = datetime.date(1986, 4, 27)

iso_widget = widgets.Dropdown(
    options=['I_131_(Bq/m3)', 'Cs_134_(Bq/m3)', 'Cs_137_(Bq/m3)'],
    value='I_131_(Bq/m3)',
    description='Isotope:',
    disabled=False
)
ui = widgets.VBox([widgets.HBox([date_widget, iso_widget])])
out = widgets.interactive_output(create_map, {'date': date_widget, 'iso_widget': iso_widget})
display(ui, out)